In [1]:
# Bibliotecas
import pandas as pd
import numpy as np
import glob

In [2]:
# Carregando os dados
# Listar os arquivos CSV na pasta 'uploads'
arquivo_csv = glob.glob('uploads/processos_*.csv')

# Carregar os arquivos CSV e concatenar em um único DataFrame
dfs = []
for arquivo in arquivo_csv:
    # Extrair o ano do nome do arquivo
    ano = int(arquivo.split('_')[-1].split('.')[0])
    df_ano = pd.read_csv(arquivo, sep=',', encoding='utf-8')
    df_ano['ano_arquivo'] = ano  # Adicionar coluna com o ano do arquivo
    dfs.append(df_ano)

df = pd.concat(dfs, ignore_index=True)

print(df.shape)

(3245632, 11)


In [3]:
# Limpeza e tratamento
# Verificar o nome correto das colunas (pode haver diferenças de acentuação ou espaços)
colunas = df.columns.tolist()

# Encontrar as colunas de data corretamente
coluna_serventia = [col for col in colunas if 'serventia' in col.lower()][0]
coluna_distribuicao = [col for col in colunas if 'data_distribuicao' in col.lower()][0]
coluna_baixa = [col for col in colunas if 'data_baixa' in col.lower()][0]
coluna_area_acao = [col for col in colunas if 'nome_area_acao' in col.lower()][0]
coluna_processo_id = [col for col in colunas if 'processo' in col.lower()][0]
coluna_comarca = [col for col in colunas if 'comarca' in col.lower()][0]

# Renomear colunas para garantir consistência
df = df.rename(columns={
coluna_distribuicao: 'data_distribuicao',
coluna_baixa: 'data_baixa',
coluna_area_acao: 'nome_area_acao',
coluna_processo_id: 'processo',
coluna_comarca: 'comarca',
coluna_serventia: 'serventia'
})

# Converter colunas de data para datetime com tratamento de erros
df['data_distribuicao'] = pd.to_datetime(df['data_distribuicao'], errors='coerce')
df['data_baixa'] = pd.to_datetime(df['data_baixa'], errors='coerce')

In [8]:
# 1. Extrair ano de distribuição e baixa
df['ano_distribuicao'] = df['data_distribuicao'].dt.year
df['ano_baixa'] = df['data_baixa'].dt.year
df['mes_distribuicao'] = df['data_distribuicao'].dt.month
df['mes_baixa'] = df['data_baixa'].dt.month

# Chave de agrupamento para as estatísticas
grouping_keys = ['comarca', 'serventia', 'nome_area_acao']

# 2. Calcular Distribuídos por ano
distribuidos_df = df.dropna(subset=['ano_distribuicao']).groupby(
    ['ano_distribuicao'] + grouping_keys
).size().reset_index(name='Distribuídos')
distribuidos_df = distribuidos_df.rename(columns={'ano_distribuicao': 'ano'})

# 2.1 Calcular Distribuídos por mês
distribuidos_mes_df = df.dropna(subset=['ano_distribuicao', 'mes_distribuicao']).groupby(
    ['ano_distribuicao', 'mes_distribuicao'] + grouping_keys
).size().reset_index(name='Distribuídos_mes')
distribuidos_mes_df = distribuidos_mes_df.rename(columns={'ano_distribuicao': 'ano', 'mes_distribuicao': 'mes'})

# 3. Calcular Baixados por ano
baixados_df = df.dropna(subset=['ano_baixa']).groupby(
    ['ano_baixa'] + grouping_keys
).size().reset_index(name='Baixados')
baixados_df = baixados_df.rename(columns={'ano_baixa': 'ano'})

# 3.1 Calcular Baixados por mês
baixados_mes_df = df.dropna(subset=['ano_baixa', 'mes_baixa']).groupby(
    ['ano_baixa', 'mes_baixa'] + grouping_keys
).size().reset_index(name='Baixados_mes')
baixados_mes_df = baixados_mes_df.rename(columns={'ano_baixa': 'ano', 'mes_baixa': 'mes'})

# 4. Calcular Pendentes por ano de distribuição
pendentes_df = df[df['data_baixa'].isna()].dropna(subset=['ano_distribuicao']).groupby(
    ['ano_distribuicao'] + grouping_keys
).size().reset_index(name='Pendentes')
pendentes_df = pendentes_df.rename(columns={'ano_distribuicao': 'ano'})

# 4.1 Calcular Pendentes por mês de distribuição
pendentes_mes_df = df[df['data_baixa'].isna()].dropna(subset=['ano_distribuicao', 'mes_distribuicao']).groupby(
    ['ano_distribuicao', 'mes_distribuicao'] + grouping_keys
).size().reset_index(name='Pendentes_mes')
pendentes_mes_df = pendentes_mes_df.rename(columns={'ano_distribuicao': 'ano', 'mes_distribuicao': 'mes'})

# 5. Juntar tudo em uma única tabela (estatisticas)
estatisticas = pd.merge(distribuidos_df, baixados_df, on=['ano'] + grouping_keys, how='outer')
estatisticas = pd.merge(estatisticas, pendentes_df, on=['ano'] + grouping_keys, how='outer')

# 5.1 Juntar tudo em uma única tabela (estatisticas_mes)
merge_keys = ['ano', 'mes'] + grouping_keys
estatisticas_mes = pd.merge(distribuidos_mes_df, baixados_mes_df, on=merge_keys, how='outer')
estatisticas_mes = pd.merge(estatisticas_mes, pendentes_mes_df, on=merge_keys, how='outer')

# 6. Limpar e organizar o DataFrame final
estatisticas = estatisticas.fillna(0)
estatisticas[['Distribuídos', 'Baixados', 'Pendentes']] = estatisticas[['Distribuídos', 'Baixados', 'Pendentes']].astype(int)

# 6.1 Limpar e organizar o DataFrame final
estatisticas_mes = estatisticas_mes.fillna(0)
estatisticas_mes[['Distribuídos_mes', 'Baixados_mes', 'Pendentes_mes']] = estatisticas_mes[['Distribuídos_mes', 'Baixados_mes', 'Pendentes_mes']].astype(int)

# Remover linhas onde o ano não pôde ser determinado
estatisticas = estatisticas.dropna(subset=['ano'])
estatisticas['ano'] = estatisticas['ano'].astype(int)

# Remover linhas onde o mês não pôde ser determinado
estatisticas_mes = estatisticas_mes.dropna(subset=['ano', 'mes'])
estatisticas_mes[['ano', 'mes']] = estatisticas_mes[['ano', 'mes']].astype(int)

# 7. Calcular a Taxa de Congestionamento por ano
soma_pend_baix = estatisticas['Pendentes'] + estatisticas['Baixados']
estatisticas['Taxa de Congestionamento (%)'] = np.where(
    soma_pend_baix > 0,
    (estatisticas['Pendentes'] / soma_pend_baix) * 100,
    0
).round(2)

# 7.1 Calcular a Taxa de Congestionamento por mês
soma_pend_baix_mes = estatisticas_mes['Pendentes_mes'] + estatisticas_mes['Baixados_mes']
estatisticas_mes['Taxa de Congestionamento_mes (%)'] = np.where(
    soma_pend_baix_mes > 0,
    (estatisticas_mes['Pendentes_mes'] / soma_pend_baix_mes) * 100,
    0
).round(2)

# 8. Ordenar e reordenar colunas para melhor visualização
estatisticas = estatisticas.sort_values(by=['ano', 'comarca', 'serventia', 'nome_area_acao'], ascending=[False, True, True, True])

colunas_finais = [
    'ano', 'nome_area_acao', 'comarca', 'serventia', 
    'Distribuídos', 'Baixados', 'Pendentes', 'Taxa de Congestionamento (%)'
]
estatisticas = estatisticas[colunas_finais]

# 8.1 Ordenar e reordenar colunas para melhor visualização
estatisticas_mes = estatisticas_mes.sort_values(by=['ano', 'mes','comarca', 'serventia', 'nome_area_acao'], ascending=[False, True, True, True, True])

colunas_finais_mes = [
    'ano', 'mes', 'nome_area_acao', 'comarca', 'serventia', 
    'Distribuídos_mes', 'Baixados_mes', 'Pendentes_mes', 'Taxa de Congestionamento_mes (%)'
]
estatisticas_mes = estatisticas_mes[colunas_finais_mes]

#print(estatisticas.head(10))


print(estatisticas_mes.head(10))


        ano  mes                    nome_area_acao    comarca  \
54007  2025    1                             civel  ABADIÂNIA   
54008  2025    1                          criminal  ABADIÂNIA   
54009  2025    1                familia - interior  ABADIÂNIA   
54010  2025    1             fazenda publica mista  ABADIÂNIA   
54011  2025    1  infancia e juventude infracional  ABADIÂNIA   
54012  2025    1            juizado especial civel  ABADIÂNIA   
54013  2025    1         juizado especial criminal  ABADIÂNIA   
54014  2025    1  juizado especial fazenda publica  ABADIÂNIA   
54015  2025    1                             civel    ACREÚNA   
54016  2025    1                familia - interior    ACREÚNA   

                                               serventia  Distribuídos_mes  \
54007                                      Vara Judicial                49   
54008                                      Vara Judicial                18   
54009                                      Vara Ju

In [ ]:
# Gera o arquivo CSV com o dataframe completo por ano:
output_filename = 'taxa_congestionamento.csv'
estatisticas.to_csv(output_filename, index=False, encoding='utf-8-sig', sep=',')

In [9]:
# Gera o arquivo CSV com o dataframe completo por mês
output_filename_mes = 'taxa_cong_mensal.csv'
estatisticas_mes.to_csv(output_filename_mes, index=False, encoding='utf-8-sig', sep=',')

In [10]:
# Gera o arquivo JSON com o dataframe completo
output_filename_json = 'tx_cong.json'
estatisticas.to_json(output_filename_json, orient='records', indent=4, force_ascii=False)

In [10]:
# Gera o arquivo JSON com o dataframe completo por mês
output_filename_json_mes = 'tx_cong_mensal.json'
estatisticas_mes.to_json(output_filename_json_mes, orient='records', indent=4, force_ascii=False)